In [ ]:
from ecell4 import *
import numpy as np
import math
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from ecell4.extra import ensemble

def wrap(job,job_id,task_id):
    def singlerun(kaf,run,correct,L,conc,shift):
        import numpy as np
        from math import sqrt
        rm = 0.005
        run+=shift
        rng = GSLRandomNumberGenerator()
        rng.seed(run)
        D = 1
        kd= 4.0*math.pi*D
        ka = kaf*kd
        f=1.0208582         
        if correct=='new':
            gamma=0.577216
            factor = (2 * sqrt(2.0) + 4 * sqrt(3.0) + 3 * sqrt(6.0) + sqrt(22.0))**2/(72 * (6 * sqrt(2.0) + 4 * sqrt(3.0) + 3 * sqrt(6.0)))            
            PP=1./(1+math.sqrt(3)*(math.log(f*f/12)+(4*math.pi*D/ka)-2*gamma)/2/math.pi)
            ka = PP*D/factor            
        voxelr=rm*f
        tm =(2*voxelr)**2/(4*D)        
        duration=0.2
        H = 3*voxelr*2
        ori = Real3(0.5*H,0,0)
        unit0 = Real3(0,0,L)
        unit1 = Real3(0,L,0)
        w = spatiocyte.SpatiocyteWorld(Real3(H,L,L),voxelr,rng)
        size=tuple(w.calculate_shape(Real3(H,L,L),voxelr))
        Nmv=size[1]*size[2]
        lx,ly,lz=w.actual_lengths()
        N=int(conc*ly*lz)
        w.add_structure(Species('M'),PlanarSurface(ori,unit0,unit1))
        with species_attributes():
            A |  {'D': str(0), 'radius':str(voxelr),'location':'M'}
            B |  {'D': str(D), 'radius':str(voxelr),'location':'M'}

        with reaction_rules():
            A + B > B | ka

        m2 = get_model()
        w.bind_to(m2)
        w.add_molecules(Species('A'), N)
        w.add_molecules(Species('B'), N)
        sim = spatiocyte.SpatiocyteSimulator(w)
        sim.initialize()
        tfactor=50
        obs = FixedIntervalNumberObserver(tm*tfactor,['A'])
        #NM = w.get_value_exact(Species('M'))
        #print('nm',NM)
        #sim.run(duration,obs)       
        #data=obs.data()
        #folder='/home/chew/outputdata/keff/2D/{}kfac{}_N{}_L{}_{}tm_{}rm/'.format(correct,kaf,N,L,tfactor,f)
        #name = folder+'run'+str(run)
        #if not os.path.exists(folder):
            #os.makedirs(folder)            
        #filename=open(name,'w')
        #np.savetxt(name,data,delimiter=',',fmt='%s')
        #filename.close()       
        if run==1:
            print('lx{},ly{},lz{}'.format(lx,ly,lz))
            print('size {},{},{}'.format(size[0],size[1],size[2]))
            print('total voxel',Nmv)
            print('N {}, rv{}, Area {}, actual Area {:.4f}'.format(N,voxelr,L*L,ly*lz))
            print('100tm',tm*100)
            print('sim.dt(),tm',sim.dt(),tm,tm/sim.dt())
        return 
    job.update({'run':task_id})
    out=singlerun(**job)
    return       

simrun=10
kaf=100
nproc=500#250#
L=5
shift=0
# works in low conc, deviate when conc is high
for conc in [4]:
    for cor in ['new']:#,'new']:
        jobs=[{'kaf':kaf,'correct':cor,'conc':conc,'L':L,'shift':shift}]    
        ensemble.run_slurm(wrap,jobs,n=simrun,nproc=nproc,path='tmp',modules=('numpy','ecell4','math'),extra_args=['--exclude=ibc[02,12,27]'])[0]
        #ensemble.run_multiprocessing(wrap,jobs,n=simrun,nproc=nproc,modules=('numpy','ecell4','math'))[0]
print('finish')

DEBUG:sge:Submitted batch job 2392309
DEBUG:sge:             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
DEBUG:sge:2392309_[1-10%500]     debug sge-4kgi     chew PD       0:00      1 (None)
DEBUG:sge:
INFO:sge:Job 2392309 must be queued, running or being transferred


In [7]:
Ns = 276774
actualArea= 24.9797
100/actualArea
L=5, Ns = 276774, actual Area (x unit area) = 24.9797


4.0032506395192895

In [11]:
print('ec3, layer{} x row {},actual vacant voxel {}, difference={}'.format(579,502,288500,579*502-288500))
print('ec4, layer{} x row {},actual vacant voxel {}, difference={}'.format(502,578,290156,502*578-290156))

ec3, layer579 x row 502,actual vacant voxel 288500, difference=2158
ec4, layer502 x row 578,actual vacant voxel 290156, difference=0


In [5]:
290156-276774

13382

In [15]:
l=0.01
l*np.array([579,502,502,578])

array([5.79, 5.02, 5.02, 5.78])

In [28]:
from math import sqrt
getA = lambda N,rv: N*rv**2*2*sqrt(3)
getA(468096,0.00510429),6.49776*6.50826

(42.247035016616685, 42.289111497600004)

In [30]:
getA(290156,0.005),5.0056*5.02

(25.128246706047758, 25.128111999999998)

In [34]:
rv=0.00510429
uA = 2*sqrt(3)*rv**2
uA*468096

42.247035016616685

In [46]:
int(2.37*6.5*6.5)

100

In [11]:
from ecell4 import *
import numpy as np
import math
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from math import sqrt,pi

kaf=100
nproc=500#250#
L=5
shift=0
conc=4.
rm = 0.005
run=1
correct='new'
rng = GSLRandomNumberGenerator()
rng.seed(run)
D = 1
kd= 4.0*math.pi*D
ka = kaf*kd
f=1.0208582         
if correct=='new':
    gamma=0.577216
    factor = (2 * sqrt(2.0) + 4 * sqrt(3.0) + 3 * sqrt(6.0) + sqrt(22.0))**2/(72 * (6 * sqrt(2.0) + 4 * sqrt(3.0) + 3 * sqrt(6.0)))            
    PP=1./(1+math.sqrt(3)*(math.log(f*f/12)+(4*math.pi*D/ka)-2*gamma)/2/math.pi)
    ka = PP*D/factor            
voxelr=rm*f
tm =(2*voxelr)**2/(4*D)        
duration=1#tm*5e4#tm*5e3
H = 3*voxelr*2
ori = Real3(0.5*H,0,0)
unit0 = Real3(0,0,L)
unit1 = Real3(0,L,0)
w = spatiocyte.SpatiocyteWorld(Real3(H,L,L),voxelr,rng)
size=tuple(w.calculate_shape(Real3(H,L,L),voxelr))
Nmv=size[1]*size[2]
lx,ly,lz=w.actual_lengths()
N=int(conc*ly*lz)
print(lx,ly,lz)
print(size[0],size[1],size[2])
#print(Nmv*(voxelr*2)**2)
w.add_structure(Species('M'),PlanarSurface(ori,unit0,unit1))
with species_attributes():
    A |  {'D': str(0), 'radius':str(voxelr),'location':'M'}
    B |  {'D': str(D), 'radius':str(voxelr),'location':'M'}

with reaction_rules():
    A + B > B | ka

m2 = get_model()
#print('ka',ka,'Rv',voxelr*2)
w.bind_to(m2)
w.add_molecules(Species('A'),2)


0.05001163379472196 5.021626286014201 5.022622343999999
6 492 568


In [18]:
for i in range(10):
    pid,p=w.list_particles(Species('M'))[i]
    print(tuple(p.position()))

(0.008335272299120327, 0.002946963782872184, 0.0051042909999999995)
(0.008335272299120327, 0.002946963782872184, 0.015312872999999998)
(0.008335272299120327, 0.002946963782872184, 0.025521455)
(0.008335272299120327, 0.002946963782872184, 0.035730037)
(0.008335272299120327, 0.002946963782872184, 0.04593861899999999)
(0.008335272299120327, 0.002946963782872184, 0.056147200999999994)
(0.008335272299120327, 0.002946963782872184, 0.06635578299999999)
(0.008335272299120327, 0.002946963782872184, 0.076564365)
(0.008335272299120327, 0.002946963782872184, 0.08677294699999999)
(0.008335272299120327, 0.002946963782872184, 0.096981529)


(0.008335272299120327, 3.6188715253670414, 1.755876104)